<a href="https://colab.research.google.com/github/mennatallah222/Fake-New-Prediction/blob/main/Fake_news_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
fake_df = pd.read_csv('/content/Fake.csv', low_memory=False)
truth_df = pd.read_csv('/content/True.csv', low_memory=False)

In [5]:
fake_df['label']=1
truth_df['label']=0

df=pd.concat([fake_df, truth_df], ignore_index=True)
df=df.sample(frac=1).reset_index(drop=True)
print(df.head())

                                               title  \
0  Some Sept. 11 families join criticism of Trump...   
1  France calls for Catalonia discussions within ...   
2  Guatemala prosecutors target ex-president for ...   
3  Obama to meet Senate leaders over Supreme Cour...   
4   Trump Has A Fantastic ‘New’ Plan To Win Back ...   

                                                text       subject  \
0  NEW YORK (Reuters) - Several relatives of vict...  politicsNews   
1  PARIS (Reuters) - France on Wednesday reiterat...     worldnews   
2  GUATEMALA CITY (Reuters) - Guatemala s attorne...     worldnews   
3  WASHINGTON (Reuters) - President Barack Obama ...  politicsNews   
4  Donald Trump is hemorrhaging Republican suppor...          News   

                 date  label  
0   January 31, 2017       0  
1   October 11, 2017       0  
2    October 6, 2017       0  
3  February 25, 2016       0  
4      August 9, 2016      1  


In [6]:
df.shape

(44898, 5)

In [16]:
rows_to_keep = len(df) // 3
df_reduced = df.sample(n=rows_to_keep, random_state=42).reset_index(drop=True)
df_reduced.shape

(14966, 5)

In [19]:
#counting the missing values
df_reduced.isnull().sum()

,0
title,0
text,0
subject,0
date,0
label,0


In [59]:
#separating the data from the target col
X=df_reduced.drop(columns='label', axis=1)
Y=df_reduced['label']

In [24]:
print(X)
print(Y)

                                                   title  \
0      White House open to small changes on corporate...   
1      Man with sword injures police outside UK Queen...   
2      Trump's tariff plan could boomerang, spark tra...   
3       GOP Senator BURNS The Hell Out Of Trump With ...   
4      GOD SQUAD: Jury Finds Polygamous Mormon Towns ...   
...                                                  ...   
14961   Republican Senator STUNS In Town Hall, Admits...   
14962  WATCH: DEMOCRAT STATE SENATOR Who Said She Wou...   
14963  BILL CLINTON GETS HAMMERED By Protesters In Ph...   
14964  U.S. Senate Democratic leader dismisses Republ...   
14965   Cops Dragged A Half-Naked Woman Into Her Cour...   

                                                    text       subject  \
0      WASHINGTON (Reuters) - The White House is will...  politicsNews   
1      (Reuters) - A man who assaulted police officer...     worldnews   
2       WASHINGTON (Reuters) - Donald Trump’s threats... 

In [25]:
port_stem=PorterStemmer()

In [26]:
def stemming_process(text):
  stemmed_text=re.sub('[^a-zA-Z]', ' ', text)
  stemmed_text=stemmed_text.lower()
  stemmed_text=stemmed_text.split()
  stemmed_text=[port_stem.stem(word) for word in stemmed_text if not word in stopwords.words('english')]
  stemmed_text=' '.join(stemmed_text)
  return stemmed_text

In [27]:
df_reduced['text']=df_reduced['text'].apply(stemming_process)

In [28]:
print(df_reduced['text'])

0        washington reuter white hous will consid small...
1        reuter man assault polic offic four foot sword...
2        washington reuter donald trump threat slap ste...
3        gop senat ben sass nebraska one trump vocal cr...
4        jami ross courthous news servicephoenix two to...
                               ...                        
14961    lot buzz gop latest effort repeal replac obama...
14962    last week mo state senat chappel nadal said ho...
14963    bill clinton philli today got swamp protest ho...
14964    washington reuter u senat democrat leader harr...
14965    louisvil kentucki metro correct judg outrag ja...
Name: text, Length: 14966, dtype: object


In [29]:
#separating the data from the target
X=df_reduced['text'].values
Y=df_reduced['label'].values

In [31]:
print(X)
print(Y)

['washington reuter white hous will consid small increas corpor tax rate need final bill u congress white hous budget chief mick mulvaney said sunday mulvaney made comment presid donald trump suggest saturday corpor tax rate could end percent senat hous repres reconcil confer respect version legisl even though bill current stand percent understand senat bill percent rate hous percent rate happi number mulvaney said interview cb face nation someth small happen confer get us across finish line look case case basi think see signific chang posit corpor tax mulvaney said'
 'reuter man assault polic offic four foot sword outsid queen elizabeth buckingham palac resid shout allahu akbar god greatest question counter terror polic saturday two unarm offic suffer slight cut detain man drove polic van friday even took sword front passeng foot well car london metropolitan polic said earli say man plan said command dean haydon head met counter terror command believ man act alon look suspect stage sa

In [32]:
Y.shape

(14966,)

In [33]:
#converting the textual data to numerical
vectorizer=TfidfVectorizer()

vectorizer.fit(X)

X=vectorizer.transform(X)

In [34]:
print(X)

  (0, 51723)	0.08832762446644396
  (0, 51563)	0.09986251881550151
  (0, 51027)	0.04779359307318235
  (0, 50186)	0.0930887228149099
  (0, 49670)	0.05484622210680915
  (0, 48998)	0.07716536549741379
  (0, 48005)	0.03505686601753964
  (0, 46907)	0.06626447549965617
  (0, 46839)	0.05202388316952598
  (0, 46153)	0.22082027382653233
  (0, 45274)	0.06985639505221722
  (0, 45179)	0.07071251873096711
  (0, 44366)	0.0664160238139381
  (0, 43724)	0.06573038811947075
  (0, 43364)	0.16143925214817098
  (0, 42856)	0.0877493523422377
  (0, 41899)	0.1118002553746121
  (0, 41778)	0.05364461943202249
  (0, 41145)	0.07467563318819165
  (0, 40722)	0.08167351424824998
  (0, 39347)	0.035733415704978684
  (0, 39243)	0.0776839872074551
  (0, 39143)	0.060324798735193426
  (0, 38631)	0.13558817239834342
  (0, 38303)	0.3122375740315407
  :	:
  (14965, 8865)	0.02364435896535212
  (14965, 8505)	0.10827865289831437
  (14965, 8391)	0.03879731292043565
  (14965, 8341)	0.0635980972790105
  (14965, 7613)	0.032603923562

In [36]:
x_train, x_test, y_train, y_test=train_test_split(X, Y, test_size=0.3, stratify=Y, random_state=3)

**Models**

In [54]:
#Logistic Regression
lr_model=LogisticRegression()
lr_model.fit(x_train, y_train)

x_train_pred=lr_model.predict(x_train)
data_accuracy=accuracy_score(x_train_pred, y_train)
print("Accuracy of data training", data_accuracy)

x_test_pred=lr_model.predict(x_test)
test_data_accuracy=accuracy_score(x_test_pred, y_test)
print("Accuracy of the test data: ", test_data_accuracy)


Accuracy of data training 0.9887361588392516
Accuracy of the test data:  0.9788418708240535


In [55]:
#Naive Bayes
nb_model=MultinomialNB()
nb_model.fit(x_train, y_train)

x_train_pred_nb=nb_model.predict(x_train)
data_accuracy_nb=accuracy_score(x_train_pred_nb, y_train)
print("Accuracy of data training", data_accuracy_nb)

x_test_pred_nb=nb_model.predict(x_test)
test_data_accuracy_nb=accuracy_score(x_test_pred_nb, y_test)
print("Accuracy of the test data: ", test_data_accuracy_nb)

Accuracy of data training 0.9452080946926308
Accuracy of the test data:  0.9325167037861916


In [58]:
#SVM
svm_model=LinearSVC(dual=False)
svm_model.fit(x_train, y_train)

x_train_pred_svm=svm_model.predict(x_train)
data_accuracy_svm=accuracy_score(x_train_pred_svm, y_train)
print("Accuracy of data training", data_accuracy_svm)

x_test_pred_svm=svm_model.predict(x_test)
test_data_accuracy_svm=accuracy_score(x_test_pred_svm, y_test)
print("Accuracy of the test data: ", test_data_accuracy_svm)

Accuracy of data training 0.9997136311569301
Accuracy of the test data:  0.987750556792873
